In [1]:
import pandas as pd 
import sklearn
import tensorflow as tf

In [2]:
# Read the CSV file
file_path = 'Data/diabetes_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df.head()

,Age,Sex,HighChol,CholCheck,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,GenHlth,MentHlth,PhysHlth,DiffWalk,Stroke,HighBP,Diabetes
0,4.0,1.0,0.0,1.0,26.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,5.0,30.0,0.0,0.0,1.0,0.0
1,12.0,1.0,1.0,1.0,26.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0
2,13.0,1.0,0.0,1.0,26.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0
3,11.0,1.0,1.0,1.0,28.0,1.0,0.0,1.0,1.0,1.0,0.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0
4,8.0,0.0,0.0,1.0,29.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Reviewing Datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   70692 non-null  float64
 1   Sex                   70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   HeartDiseaseorAttack  70692 non-null  float64
 7   PhysActivity          70692 non-null  float64
 8   Fruits                70692 non-null  float64
 9   Veggies               70692 non-null  float64
 10  HvyAlcoholConsump     70692 non-null  float64
 11  GenHlth               70692 non-null  float64
 12  MentHlth              70692 non-null  float64
 13  PhysHlth              70692 non-null  float64
 14  DiffWalk              70692 non-null  float64
 15  Stroke             

In [4]:
import os
import pandas as pd
import sqlite3
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Adjust the file path 
file_path = os.path.join(os.getcwd(), 'Data', 'diabetes_data.csv')  # Assuming the script is in the parent folder of "Data"
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Dataset file not found at {file_path}. Ensure the file is in the 'Data' folder.")

# Load the dataset
diabetes_data = pd.read_csv(file_path)

# Rename column to match schema if necessary
if 'HvyAlcoholConsump' in diabetes_data.columns:
    diabetes_data.rename(columns={'HvyAlcoholConsump': 'hvyalcohol'}, inplace=True)

# Create a SQLite database and load the data
db_connection = sqlite3.connect('diabetes_project.db')
cursor = db_connection.cursor()

schema_sql = """
DROP TABLE IF EXISTS diabetes;

CREATE TABLE diabetes (
    age FLOAT,
    sex FLOAT,
    highchol FLOAT,
    cholcheck FLOAT,
    bmi FLOAT,
    smoker FLOAT,
    heartdiseaseorattack FLOAT,
    physactivity FLOAT,
    fruits FLOAT,
    veggies FLOAT,
    hvyalcohol FLOAT,
    genhlth FLOAT,
    menthlth FLOAT,
    physhlth FLOAT,
    diffwalk FLOAT,
    stroke FLOAT,
    highbp FLOAT,
    diabetes FLOAT
);
"""

cursor.executescript(schema_sql)
db_connection.commit()

# Load data into the SQLite database
diabetes_data.to_sql('diabetes', db_connection, if_exists='append', index=False)

# Fetch the data back from SQLite
query = "SELECT * FROM diabetes"
diabetes_df = pd.read_sql_query(query, db_connection)
db_connection.close()

# Separate features (X) and target variable (y)
X = diabetes_df.drop(columns=['diabetes'])
y = diabetes_df['diabetes']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim = len(X_train_scaled[0])
node_value1 = 20
node_value2 = 10
node_value3 = 1
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=node_value1, activation="relu", input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=node_value2, activation="relu",))

# Output layer
nn.add(tf.keras.layers.Dense(units=node_value3, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")






c:\Users\micah\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 20)             │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 581 (2.27 KB)

 Trainable params: 581 (2.27 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 2s 678us/step - accuracy: 0.6926 - loss: 0.5656
Epoch 2/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 629us/step - accuracy: 0.7511 - loss: 0.5077
Epoch 3/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 655us/step - accuracy: 0.7490 - loss: 0.5068
Epoch 4/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 648us/step - accuracy: 0.7545 - loss: 0.5035
Epoch 5/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 633us/step - accuracy: 0.7536 - loss: 0.5007
Epoch 6/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 651us/step - accuracy: 0.7537 - loss: 0.5033
Epoch 7/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 631us/step - accuracy: 0.7508 - loss: 0.5027
Epoch 8/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 639us/step - accuracy: 0.7578 - loss: 0.4976
Epoch 9/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 649us/step - accuracy: 0.7540 - loss: 0.5042
Epoch 10/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 627us/step - accuracy: 0.7540 - loss: 0.5042
Epoch 11/100
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 1s 646us/step - accuracy: 0.7534 - loss: 0.50